In [ ]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext training_ml_control
%set_random_seed 12

In [ ]:
%presentation_style

In [ ]:
%load_latex_macros

In [ ]:
%autoreload
import os
import warnings
from dataclasses import dataclass
from typing import Protocol

import casadi
import do_mpc
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from gymnasium import Env
from IPython.display import HTML
from ipywidgets import interact, widgets
from matplotlib.animation import FuncAnimation
from numpy.typing import NDArray

from training_ml_control.environment import (
    create_inverted_pendulum_environment,
)

from training_ml_control.shortest_path_problem import (
    create_shortest_path_graph,
    plot_shortest_path_graph,
    plot_all_paths_graph,
)

from training_ml_control.plots import (
    create_inverted_pendulum_environment,
    plot_inverted_pendulum_results,
    animate_mass_spring_damper_simulation,
    animate_inverted_pendulum_simulation,
    animate_full_inverted_pendulum_simulation,
)

from training_ml_control.nb_utils import (
    display_array,
    show_video,
)

warnings.simplefilter("ignore", UserWarning)
sns.set_theme()
plt.rcParams["figure.figsize"] = [9, 5]
# This is needed because inside docker the rendering of mujoco environments may not work.
render_mode = "rgb_array" if os.environ.get("DISPLAY") else None

### Linearized Model

In [ ]:
env = create_inverted_pendulum_environment()

ip_parameters = InvertedPendulumParameters(
    l=env.model.geom_pos[2, 2],
    m=env.model.body_mass[2],
    M=env.model.body_mass[1],
)
# Dynamics matrix
A = np.array(
    [
        [0, 1],
        [
            (ip_parameters.M + ip_parameters.m)
            * ip_parameters.g
            / (ip_parameters.m * ip_parameters.l),
            0,
        ],
    ]
)
# Input matrix
B = np.array(
    [[0, -ip_parameters.gamma / (ip_parameters.M * ip_parameters.l)]]
).transpose()
# Output matrices
C = np.array([[1, 0], [0, 1]])
D = np.zeros(2)

inverted_pendulum_lin = do_mpc.model.LinearModel("continuous")
theta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="theta")
dtheta = inverted_pendulum_lin.set_variable(var_type="_x", var_name="dtheta")
inverted_pendulum_lin.set_variable(var_type="_u", var_name="force")

# Inertia
J = (ip_parameters.m * ip_parameters.l**2) / 3
# Energies
E_kin = 0.5 * J * dtheta**2 + 0.5 * ip_parameters.m * (
    (ip_parameters.l * dtheta * casadi.cos(theta)) ** 2
    + (ip_parameters.l * dtheta * casadi.sin(theta)) ** 2
)
E_pot = ip_parameters.m * ip_parameters.g * ip_parameters.l * casadi.cos(theta)
inverted_pendulum_lin.set_expression("E_kinetic", E_kin)
inverted_pendulum_lin.set_expression("E_potential", E_pot)

inverted_pendulum_lin.setup(A, B, C, D)
env = create_inverted_pendulum_environment()
inverted_pendulum_lin = inverted_pendulum_lin.discretize(env.dt)

### Non-Linear Model

In [ ]:
inverted_pendulum = do_mpc.model.Model("continuous")

pos = inverted_pendulum.set_variable(var_type="_x", var_name="position")
theta = inverted_pendulum.set_variable(var_type="_x", var_name="theta")
dpos = inverted_pendulum.set_variable(var_type="_x", var_name="velocity")
dtheta = inverted_pendulum.set_variable(var_type="_x", var_name="dtheta")
u = inverted_pendulum.set_variable(var_type="_u", var_name="force")

k = 1 / 3
numerator = (
    (ip_parameters.M + ip_parameters.m) * ip_parameters.g * casadi.sin(theta)
    - casadi.cos(theta)
    * (
        ip_parameters.gamma * u
        + ip_parameters.m * ip_parameters.l * dtheta**2 * casadi.sin(theta)
        - ip_parameters.mu_c * dpos
    )
    - (ip_parameters.M + ip_parameters.m)
    / (ip_parameters.m * ip_parameters.l)
    * ip_parameters.mu_p
    * dtheta
)
denominator = (1 + k) * (
    ip_parameters.M + ip_parameters.m
) * ip_parameters.l - ip_parameters.m * ip_parameters.l * casadi.cos(theta) ** 2
ddtheta = numerator / denominator

numerator = (
    ip_parameters.m * ip_parameters.g * casadi.cos(theta) * casadi.sin(theta)
    - (1 + k)
    * (
        ip_parameters.gamma * u
        + ip_parameters.m * ip_parameters.l * dtheta**2 * casadi.sin(theta)
        - ip_parameters.mu_c * dpos
    )
    - ip_parameters.mu_p * dtheta * casadi.cos(theta) / ip_parameters.l
)
denominator = ip_parameters.m * casadi.cos(theta) ** 2 - (1 + k) * (
    ip_parameters.M + ip_parameters.m
)
ddpos = numerator / denominator

inverted_pendulum.set_rhs("position", dpos)
inverted_pendulum.set_rhs("theta", dtheta)
inverted_pendulum.set_rhs("velocity", ddpos)
inverted_pendulum.set_rhs("dtheta", ddtheta)

# Inertia
J = (ip_parameters.m * ip_parameters.l**2) / 3
# Energies
E_kin = (
    0.5 * J * dtheta**2
    + 0.5 * ip_parameters.M * dpos**2
    + 0.5
    * ip_parameters.m
    * (
        (dpos + ip_parameters.l * dtheta * casadi.cos(theta)) ** 2
        + (ip_parameters.l * dtheta * casadi.sin(theta)) ** 2
    )
)
E_pot = ip_parameters.m * ip_parameters.g * ip_parameters.l * casadi.cos(theta)
inverted_pendulum.set_expression("E_kinetic", E_kin)
inverted_pendulum.set_expression("E_potential", E_pot)

inverted_pendulum.setup()

### Simulators

In [ ]:
env = create_inverted_pendulum_environment()
linear_simulator = do_mpc.simulator.Simulator(inverted_pendulum_lin)
linear_simulator.set_param(t_step=env.dt)
linear_simulator.setup()

env = create_inverted_pendulum_environment(cutoff_angle=np.inf)
nonlinear_simulator = do_mpc.simulator.Simulator(inverted_pendulum)
params_simulator = {
    # Note: cvode doesn't support DAE systems.
    "integration_tool": "idas",
    "abstol": 1e-8,
    "reltol": 1e-8,
    "t_step": env.dt,
}
nonlinear_simulator.set_param(**params_simulator)

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::

# Model Predictive Control

Unfortunately, the analytically convenient linear quadratic problem formulations are often not satisfactory. There are two main reasons for this:

- The system may be nonlinear, and it may be inappropriate to use for
  control purposes. Moreover, some of the control variables may be naturally
  discrete, and this is incompatible with the linear system viewpoint.
  
- There may be control and/or state constraints, which are not handled
  adequately through quadratic penalty terms in the cost function. For
  example, the motion of a car may be constrained by the presence of
  obstacles and hardware limitations.
  The solution obtained from a linear quadratic model may not be suitable for such
  a problem, because quadratic penalties treat constraints "softly"
  and may produce trajectories that violate the constraints.

Model Predictive Control (MPC) is a control algorithm based on solving an **on-line** optimal control problem. A receding horizon approach is used which can be summarized in the following steps:

1. At time $t$ and for the current state $x_t$, solve, on-line, an open-loop optimal control problem over some future interval taking account the current and future constraints.
2. Apply the first step in the optimal control sequence.
3. Repeat the procedure at time $t + 1$ using the current state $x_{t + 1}$.

:::{figure} _static/images/40_model_predictive_control_horizon.svg
Illustration of the problem solved by MPC at state $x_k$.
We minimize the cost function over the next $l$ stages.
We then apply the control of the optimizing sequence up to the control horizon.
In most cases, the control horizon is set to 1.
:::

:::{figure} _static/images/40_mpc_block_diagram.svg
MPC Block Diagram.
:::

A major difference between MPC and finite-state stochastic control problems
that are popular in the RL/artificial intelligence literature is that in MPC
the state and control spaces are continuous/infinite, such as for example
in self-driving cars, the control of aircraft and drones, or the operation of
chemical processes

## Iterative Linear Quadratic Regulator

Iterative Linear Quadratic Regulator (iLQR) is an extension of LQR control to non-linear system with non-linear quadratic costs.

The idea is to approximate the cost and dynamics as quadratic and affine, respectively, then exactly solve the resulting LQR problem.

Given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a first-order Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}{lll}
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t\\
\delta\mathbf{x}_{t+1} &= A_t\delta\mathbf{x}_t + B_t\delta\mathbf{u}_t
\end{array}
$$

with
$A_t = A(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}$ and
$B_t = B(\mathbf{x}_t, \mathbf{u}_t) = \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}$

Given a general cost function that is not linear-quadratic, we use a second-order Taylor Series Expansion to linearize the dynamics into a form common for optimal control problems:

$$
\begin{array}{lll}
J_N(\mathbf{x}_0, \mathbf{U}) &=
l_f(\mathbf{x}_N) + \sum \limits_{t=1}^{N-1} l(\mathbf{x}_t, \mathbf{u}_t)\\
&\approx \frac{1}{2} \mathbf{x}^T_N Q \mathbf{x}_N + q_N^T \mathbf{x}_N
+ \sum \limits_{t=1}^{N-1}
\frac{1}{2} \mathbf{x}^T_t Q_t \mathbf{x}_t
+ \frac{1}{2} \mathbf{u}^T_t R_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{x}^T_t H_t \mathbf{u}_t
+ \frac{1}{2} \mathbf{u}^T_t H_t^T \mathbf{x}_t
+ q_t^T \mathbf{x}_t +  + r_t^T \mathbf{u}_t
\end{array}
$$

We define the following variables for convenience:

$$
\begin{array}{lll}
l_x &:= \frac{\partial l}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t \mathbf{x}_t + q_t
\\
l_u &:= \frac{\partial l}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t \mathbf{u}_t + r_t
\\
l_{xx} &:= \frac{\partial^2 l}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = Q_t
\\
l_{uu} &:= \frac{\partial^2 l}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t} = R_t
\\
l_{xu} &:= \frac{\partial^2 l}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t
\\
l_{ux} &:= \frac{\partial^2 l}{\partial \mathbf{u}\mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t} = H_t^T
\end{array}
$$

We can apply Bellman's Principle of Optimality to define the optimal cost-to-go:

$$
\begin{array}{lll}
V_N(\mathbf{x}_N) &= g_f(\mathbf{x}_N)\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u {g(\mathbf{x}_{t}, \mathbf{u}_{t}) + V_{t+1}(f(\mathbf{x}_{t}, \mathbf{u}_{N-t}))}\\
V_{t}(\mathbf{x}_t) &= \displaystyle \min_u Q_{t}(\mathbf{x}_{t}, \mathbf{u}_{t})
\end{array}\\
$$

The $Q$-function is the discrete-time analogue of the Hamiltonian, sometimes known as the pseudo-Hamiltonian.

We approximate the cost-to-go function as locally quadratic near the nominal trajectory gives us (we drop the time index in some of the equations for the sake of readability):

$$
\delta V(\mathbf{x}) = s^T \delta\mathbf{x} + \frac{1}{2} \delta\mathbf{x}^T S \delta\mathbf{x}
$$

with 
$s = \frac{\partial V}{\partial \mathbf{x}}|_{\mathbf{x}},
S = \frac{\partial^2 V}{\partial \mathbf{x}^2}|_{\mathbf{x}}$

Similarily:

$$
\delta Q(\mathbf{x}, \mathbf{u}) = 
\frac{1}{2}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}^T
\begin{bmatrix} Q_{xx} & Q_{xu} \\ Q_{ux} & Q_{uu} \end{bmatrix}
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
+
\begin{bmatrix} Q_{x} \\ Q_{u} \end{bmatrix}^T
\begin{bmatrix} \delta\mathbf{x} \\ \delta\mathbf{u} \end{bmatrix}
$$

with:

$$
\begin{array}{lll}
Q_x &= l_x + s_{t+1} A_t \\
Q_u &= l_u + s_{t+1} B_t \\
Q_{xx} &= l_{xx} + A_t^T S_{t+1} A_t \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t
\end{array}
$$

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}{lll}
\Delta V &= \frac{1}{2} k^T Q_{uu} k + k^T Q_u\\
s &= Q_x + K^T Q_{uu} k + K^T Q_u + Q_{ux}^T k\\
S &= Q_{xx} + K^T Q_{uu} K + K^T Q_{ux} + Q_{ux}^T K
\end{array}
$$

Once these terms are computed, a forward pass computes a new trajectory:

$$
\begin{array}{lll}
\hat{x}_0 &= x_0\\
\hat{u}_t &= u_t + k_t + K_t(\hat{x}_t - x_t)\\
\hat{x}_{t+1} &= f(\hat{x}_t, \hat{u}_t)
\end{array}
$$

The basic flow of the algorithm is:

1. Initialize with initial state $\mathbf{x}_0$ and initial control sequence $\mathbf{U} = \{\mathbf{u}_{0}, \mathbf{u}_{1}, \dots, \mathbf{u}_{N-1}\}$.
2. Do a forward pass using the non-linear dynamics, i.e. simulate the system using $(\mathbf{x}_0, \mathbf{U})$ to get the trajectory through state space, $\mathbf{X}$, that results from applying the control sequence $\mathbf{X}$ starting in $\mathbf{x}_0$.
3. Do a backward pass, estimate the value function and dynamics for each $(\mathbf{x}, \mathbf{u})$ in the state-space and control signal trajectories.
4. Calculate an updated control signal $\hat{\mathbf{U}}$ and evaluate cost of trajectory resulting from $(\mathbf{x}_0, \hat{\mathbf{U}})$.
   
   1. If $|(\textrm{cost}(x_0, \hat{\textbf{U}}) - \textrm{cost}(x_0, \textbf{U})| < \textrm{threshold},$ then we've converged and exit.
   2. If $\textrm{cost}(x_0, \hat{\textbf{U}}) < \textrm{cost}(x_0, \textbf{U}),$ then set $\textbf{U} = \hat{\textbf{U}},$ and change the update size to be more aggressive. Go back to step 2.
   3. If $\textrm{cost}(x_0, \hat{\textbf{U}}) \geq \textrm{cost}(x_0, \textbf{U}),$ change the update size to be more modest. Go back to step 3.

## Differential Dynamic Programming

Differential Dynamic Programming (DDP) is almost identical to iLQR but unlike the latter, it expands the system's dynamics to the second order i.e. for a given a discrete-time non-linear system with state-space representation: 

$$
\mathbf{x}_{t+1} = f(\mathbf{x}_t, \mathbf{u}_t)
$$

we approximate it with a **second-order** Taylor-series expansion about nominal trajectories
$\mathbf{X} = \{\mathbf{x}_0, \dots, \mathbf{x}_N\}, \mathbf{U} = \{ \mathbf{u}_0, \dots, \mathbf{u}_N \}$:

$$
\begin{array}{lll}
\mathbf{x}_{t+1} + \delta\mathbf{x}_{t+1} &= f(\mathbf{x}_t + \delta \mathbf{x}_t, \mathbf{u}_t + \delta \mathbf{u}_t) \\ 
& \approx f(\mathbf{x}_t, \mathbf{u}_t)
+ \frac{\partial f}{\partial \mathbf{x}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t
+ \frac{\partial f}{\partial \mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t
+ \frac{1}{2} \left(
\frac{\partial^2 f}{\partial \mathbf{x}^2}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t^2
+ 2 \frac{\partial^2 f}{\partial \mathbf{x}\mathbf{u}}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{x}_t\delta\mathbf{u}_t
+ \frac{\partial^2 f}{\partial \mathbf{u}^2}|_{\mathbf{x}_t, \mathbf{u}_t}\delta\mathbf{u}_t^2
\right)
\\
\delta\mathbf{x}_{t+1} &= f_x\delta\mathbf{x}_t + f_u\delta\mathbf{u}_t + \frac{1}{2} \left(
f_{xx} \delta\mathbf{x}_t^2 + 2 f_{xu} \delta\mathbf{x}_t \delta\mathbf{u}_t + f_{uu} \delta\mathbf{u}_t^2
\right)
\end{array}
$$

Which leads to a different expansion of the $Q$-function:

$$
\begin{array}{lll}
Q_x &= l_x + f_x^T s_{t+1} \\
Q_u &= l_u + f_u^T s_{t+1} \\
Q_{xx} &= l_{xx} + f_x^T S_{t+1} f_x + S_{t+1} \cdot f_{xx} \\
Q_{uu} &= l_{uu} + B_t^T S_{t+1} B_t + S_{t+1} \cdot f_{xu} \\
Q_{ux} &= l_{ux} + B_t^T S_{t+1} A_t + S_{t+1} \cdot f_{uu} 
\end{array}
$$

The difference with iLQR is that the last term in the last 3 equations is the product of a vector with a tensor. 

The optimal control modification $\delta\mathbf{u}^∗$ for some state perturbation $\delta\mathbf{x}$, is obtained by minimizing the quadratic model:

$$
\delta\mathbf{u}^∗(\delta\mathbf{x}) = \displaystyle\arg\min_{\delta\mathbf{u}} Q(\delta\mathbf{x}, \delta\mathbf{u}) = k + K\delta\mathbf{x}
$$

This is a locally-linear feedback policy with:

$$
k := -Q_{uu}^{-1}Q_u\\
K := -Q_{uu}^{-1}Q_{ux}
$$

Plugging this back into the expansion of $Q$, a quadratic model of $V$ is obtained. After simplification it is:

$$
\begin{array}{lll}
\Delta V &= -\frac{1}{2} k^T Q_{uu} k\\
s &= Q_x - K^T Q_{uu} k\\
S &= Q_{xx} - K^T Q_{uu} K
\end{array}
$$

### Mass-Spring-Damper

[do-mpc](https://www.do-mpc.com/en/latest/) uses [CasADi](https://web.casadi.org/python-api/) (an open-source tool for nonlinear optimization and algorithmic differentiation) for the modeling part and for the different cost functions. Here's a table of useful operators:

| Operator | Description |Equation |
| --- | --- | --- |
| [casadi.sumsqr(x)](https://web.casadi.org/python-api/#casadi.casadi.sumsqr) | Squared-sum | $\sum x_i^2$ |
| [casadi.norm_2(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_2) | $L_2$-norm | $\sqrt{\sum x_i^2}$ |
| [casadi.norm_1(x)](https://web.casadi.org/python-api/#casadi.casadi.norm_1) | $L_1$-norm | $\sum |x_i|$ |
| [casadi.bilin(A, x)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Quadratic Form | $x^T A x$ |
| [casadi.bilin(A, x, y)](https://web.casadi.org/python-api/#casadi.casadi.bilin) | Bilinear Form | $x^T A y$ |

#### Controller

First, we create an instance of the MPC class is generated with the Mass-Spring-Damper prediction model defined above.

In [ ]:
mpc = do_mpc.controller.MPC(mass_spring_damper)

We choose the finite prediction horizon `n_horizon = 20`, the time step `t_step = 0.04s` to be the same as the environment's time step. We also set the parameters of the applied discretization scheme orthogonal collocation.

In [ ]:
env = create_mass_spring_damper_environment()
mpc_params = {
    "n_horizon": 20,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

The control objective is to move the mass to a desired position (`0.1`) and keep it there.

In [ ]:
xss = np.array([0.1, 0.0])
distance_cost = 100 * casadi.norm_2(mass_spring_damper.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

We also restrict the input force.

In [ ]:
force_penalty = 1e-2
mpc.set_rterm(force=force_penalty)

#### Constraints

We apply constraints on the force. In this case, there is only an upper and lower bounds for the force.

In [ ]:
# lower and upper bounds of the input
u_max = 20
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

We can now setup the controller.

In [ ]:
mpc.setup()

#### Simulation

We set the initial state and simulate the closed-loop for 100 steps.

In [ ]:
%%capture
mpc.reset_history()
mass_spring_damper_simulator.reset_history()
x0 = np.zeros((2, 1))
mass_spring_damper_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = mass_spring_damper_simulator.make_step(u0)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

#### Evaluation

Finally, we evaluate the controller on the actual environment.

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 100
env = create_mass_spring_damper_environment(
    render_mode=render_mode, max_steps=max_steps
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_mass_spring_damper_simulation(mpc.data)

## Exercise

- Design an MPC controller for the linearized inverted pendulum.
- For each case, try different cost functions:
  - $\sum \theta^2$
  - $\sum |\theta|$
  - $E_{\text{kinetic}} - E_{\text{potential}}$

## Solution

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum_lin)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 50,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
xss = np.array([0.0, 0.0])
distance_cost = casadi.bilin(np.diag([100, 1]), inverted_pendulum_lin.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 1e-4
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
inverted_pendulum_lin_simulator.reset_history()
x0 = np.zeros((2, 1))
x0[0] = 0.01
inverted_pendulum_lin_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = inverted_pendulum_lin_simulator.make_step(u0)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(2)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation[[1, 3]].reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(render_mode=render_mode, max_steps=max_steps)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_inverted_pendulum_simulation(mpc.data)

## Exercise

- Design an MPC Controller for the non-linear inverted pendulum system for two different cases:
  1. Cart at origin and upright Pendulm: Set the reference for the cart position to the origin.
  2. Pendulum Swing-up: Set the initial angle to to $-\pi$ i.e. start with the pendulum at the bottom.
  3. Same as 2 but set the reference for the cart position to the origin.
  4. Make the pendulum rotate as fast as possible.
  
> **Note 1** Use the following to create the environment with initial angle set to -np.pi and cutoff angle to np.inf for second > case.
> ```python
> env = create_inverted_pendulum_environment(cutoff_angle=np.inf, initial_angle=-np.pi)
> ```

> **Note 2**: You can access the inverted pendulum's kinetic, respectively potential, energy using
> `inverted_pendulum.aux["E_kinetic"]`, respectively `inverted_pendulum.aux["E_potential"]`

## Solution

### Swing-up

#### Controller

In [ ]:
mpc = do_mpc.controller.MPC(inverted_pendulum)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 100,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
energy_cost = inverted_pendulum.aux["E_kinetic"] - inverted_pendulum.aux["E_potential"]
terminal_cost = energy_cost
stage_cost = energy_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 0.1
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the position
x_max = 1
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Setup

In [ ]:
mpc.setup()

#### Simulation

In [ ]:
%%capture
mpc.reset_history()
inverted_pendulum_simulator.reset_history()
x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
inverted_pendulum_simulator.x0 = x0
mpc.x0 = x0
mpc.set_initial_guess()
for k in range(100):
    u0 = mpc.make_step(x0)
    x0 = inverted_pendulum_simulator.make_step(u0)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

#### Evaluation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.array([0.0, -0.99 * np.pi, 0.0, 0.0])
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 500
env = create_inverted_pendulum_environment(
    render_mode=render_mode,
    max_steps=max_steps,
    cutoff_angle=np.inf,
    initial_angle=-np.pi,
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

##  Multi-Stage MPC

The basic idea for the multi-stage approach is to consider various scenarios, where a scenario is defined by one possible realization of all uncertain parameters at every control instant within the horizon. The family of all considered discrete scenarios can be represented as a tree structure, called the scenario tree

:::{figure} ./_static/images/40_multi_state_mpc.png
:width: 60%
:align: center
Scenario tree representation of the uncertainty
evolution for multi-stage MPC.
:::

- Each node in the tree denotes the possible state of the system at every prediction step.
- The branches represent the different possible realizations of the uncertainty.
- The initial state of the system forms the root node of the tree.
- The root node branches into several nodes in the first stage depending on the number of vertex matrix pairs of the parametric uncertainty.
- All the nodes in the first stage branch again in the second stage.
- The sequence continues until the end of prediction horizon N to form the complete scenario tree.
- A path from the root node to the leaf node represents a scenario.

## Example - Inverted Pendulum

In a real system, usually the model parameters cannot be determined exactly, what represents an important source of uncertainty. In this example, we consider that the mass of the pendulum and that of the cart are not known precisely 
and vary with respect to their nominal value.

### Model

#### Model, States and Control inputs

In [ ]:
model = do_mpc.model.Model("continuous")

pos = model.set_variable(var_type="_x", var_name="position")
theta = model.set_variable(var_type="_x", var_name="theta")
dpos = model.set_variable(var_type="_x", var_name="velocity")
dtheta = model.set_variable(var_type="_x", var_name="dtheta")
u = model.set_variable(var_type="_u", var_name="force")

#### Parameters

In [ ]:
# Certain parameters
ip_parameters = InvertedPendulumParameters()
k = 1 / 3
l = ip_parameters.l
gamma = ip_parameters.gamma
g = ip_parameters.g
mu_p = ip_parameters.mu_p
mu_c = ip_parameters.mu_c

In [ ]:
# Uncertain parameters
m = model.set_variable("_p", "m")
M = model.set_variable("_p", "M")

#### ODE

In [ ]:
numerator = (
    (M + m) * g * casadi.sin(theta)
    - casadi.cos(theta)
    * (gamma * u + m * l * dtheta**2 * casadi.sin(theta) - mu_c * dpos)
    - (M + m) / (m * l) * mu_p * dtheta
)
denominator = (1 + k) * (M + m) * l - m * l * casadi.cos(theta) ** 2
ddtheta = numerator / denominator

In [ ]:
numerator = (
    m * g * casadi.cos(theta) * casadi.sin(theta)
    - (1 + k) * (gamma * u + m * l * dtheta**2 * casadi.sin(theta) - mu_c * dpos)
    - mu_p * dtheta * casadi.cos(theta) / l
)
denominator = m * casadi.cos(theta) ** 2 - (1 + k) * (M + m)
ddpos = numerator / denominator

In [ ]:
model.set_rhs("position", dpos)
model.set_rhs("theta", dtheta)
model.set_rhs("velocity", ddpos)
model.set_rhs("dtheta", ddtheta)

#### Setup

In [ ]:
model.setup()

### Controller

In [ ]:
mpc = do_mpc.controller.MPC(model)

In [ ]:
env = create_inverted_pendulum_environment()
mpc_params = {
    "n_horizon": 50,
    "n_robust": 1,
    "t_step": env.dt,
    "state_discretization": "collocation",
    "collocation_type": "radau",
    "collocation_deg": 3,
    "collocation_ni": 1,
    "store_full_solution": True,
    # Use MA27 linear solver in ipopt for faster calculations:
    "nlpsol_opts": {"ipopt.linear_solver": "mumps"},
}
mpc.set_param(**mpc_params)

#### Objective

In [ ]:
env = create_inverted_pendulum_environment()
xss = np.array([0.5, 0, 0, 0])
distance_cost = casadi.bilin(np.diag([1, 100, 0, 0]), model.x.cat - xss)
terminal_cost = distance_cost
stage_cost = distance_cost
print(f"{stage_cost=}")
print(f"{terminal_cost=}")
mpc.set_objective(mterm=terminal_cost, lterm=stage_cost)

In [ ]:
force_penalty = 0.1
mpc.set_rterm(force=force_penalty)

#### Constraints

In [ ]:
# lower and upper bounds of the position
x_max = 1
mpc.bounds["lower", "_x", "position"] = -x_max
mpc.bounds["upper", "_x", "position"] = x_max
# lower and upper bounds of the input
u_max = 3
mpc.bounds["lower", "_u", "force"] = -u_max
mpc.bounds["upper", "_u", "force"] = u_max

#### Parameter Uncertainty

In [ ]:
m_values = ip_parameters.m * np.array([1.0, 1.30, 0.70])
M_values = ip_parameters.M * np.array([1.0, 1.30, 0.70])
mpc.set_uncertainty_values(m=m_values, M=M_values)

#### Setup

In [ ]:
mpc.setup()

### Simulation

In [ ]:
class MPCController:
    def __init__(self, mpc: do_mpc.controller.MPC) -> None:
        self.mpc = mpc
        self.mpc.reset_history()
        self.mpc.x0 = np.zeros(4)
        self.mpc.set_initial_guess()

    def act(self, observation: NDArray) -> NDArray:
        return mpc.make_step(observation.reshape(-1, 1)).ravel()

In [ ]:
%%capture
max_steps = 100
env = create_inverted_pendulum_environment(
    render_mode=render_mode,
    max_steps=max_steps,
    cutoff_angle=np.inf,
    initial_angle=0.99 * np.pi,
)
controller = MPCController(mpc)
results = simulate_environment(env, max_steps=max_steps, controller=controller)

In [ ]:
show_video(results.frames, fps=1 / env.dt)

In [ ]:
animate_full_inverted_pendulum_simulation(mpc.data)

:::{figure} ./_static/images/aai-institute-cover.png
:width: 90%
:align: center
---
name: aai-institute
---
:::